# Chapter 12: Information Analysis

## Summary
- Information analysis is a two-step process
- Step 1 is to turn information into portfolios
- Step 2 is to analyse the performance of those portfolios
- Event studies provide analysis when information arrives episodically

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Information & Active Management

- Active Managers use information to predict the future exceptional return on a group of stock
- The emphasis is on predicting _alpha_ or residual return, beta-adjusted return relative to a benchmark
- We call a set of data a _predictor_
- Any predictor is made up of signal plus noise
- Information analysis is an effort to find the signal-to-noise ratio
- Predictors are limited only by availability and imagination

**Information Classification Dimensions**
- Primary or processed
- Judgemental or impartial
- Ordinal or cardinal
- Historical, contemporary, or forecast


## Step 1: Information into Portfolios ##

**Some possible approaches**
- _Procedure 1_ &emsp; With buy/sell recommendations we could equal- (or value-) weight the buy group and the sell group
- _Procedure 2_ &emsp; With scores, we can build a portfolio for each score by equal- (or value-) weighting within each score category
- _Procedure 3_ &emsp; With straight alphas we can split the stocks into tw groups (higher than average, lower than average alphas)  We can then weight the stocks by how far their alpha is from the average
- _Procedure 4_ &emsp; With straight alphas we can rank the assets according to alpha and then group into e.g. quintiles
- _Procedure 5_ &emsp; With any numerical score we can build a factor portfolio that bets on the prediction and does not make a market bet.  Factor portfolio consists of a long portfolio and a short portfolio (of equal value and beta but the long portfolio will have a unit bet on the prediction relative to the short portfolio)
- _Procedure 6_ &emsp; With any numerical score we can build a factor portfolio (as above) designed so that the long and short portfolios are matched on a set of pre-specified control variables.  e.g. we could ensure the long and short portfolios match on industry, sector etc